In [1]:
import os

import numpy as np
import torch
from torch.utils.data import DataLoader
from lib.config import cfg
from lib.config import update_config

import lib.dataset.dataset as dataUtils
from lib.vortex.modules.efficienttrack.efficienttrack import EfficientTrack

In [2]:
training_params = {'batch_size': cfg.EFFICIENTTRACK.BATCH_SIZE,
                   'shuffle': True,
                   'drop_last': True,
                   'collate_fn': dataUtils.collater_keypoints,
                   'num_workers': cfg.DATALOADER_NUM_WORKERS,
                   'pin_memory': False}

val_params = {'batch_size': cfg.EFFICIENTTRACK.BATCH_SIZE,
              'shuffle': False,
              'drop_last': True,
              'collate_fn': dataUtils.collater_keypoints,
              'num_workers': cfg.DATALOADER_NUM_WORKERS,
              'pin_memory': False}

training_set = dataUtils.VortexDataset(cfg, set='val', mode = 'keypoints')
val_set = dataUtils.VortexDataset(cfg, set='val',mode = 'keypoints')

training_generator = DataLoader(training_set, **training_params)
val_generator = DataLoader(val_set, **val_params)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [3]:
efficientTrack = EfficientTrack('train', cfg, None)

[Info] initializing weights...


In [4]:
efficientTrack.train(training_generator, val_generator, 100)

  0%|          | 0/72 [00:00<?, ?it/s]

/home/trackingsetup/Documents/Vortex/lib/vortex/modules/efficienttrack/darkpose.py:84: RuntimeWarning: invalid value encountered in float_scalars
  hm[i,j] *= origin_max / np.max(hm[i,j])


KeyboardInterrupt: 

In [ ]:
import cv2
efficientDet = EfficientDet('inference', cfg, '/home/timo/Documents/Vortex/model_savefiles_combined/handPose_GN_2/efficientdet-d0_83_14112.pth')

out_dict = efficientDet.predict_on_image('/home/timo/Documents/CombiNet/datasets/handPose3D/train/Object_0/Camera_T/Frame_2096.jpg')
efficientDet.visualize_prediction(out_dict)

In [ ]:
efficientDet = EfficientDet('export', cfg, None)
efficientDet.export_to_onnx('test.onnx', input_size = 256)
